In [0]:
USE CATALOG `nokia-assginment-catalog`;

## Query 1: Total Number of Patents


In [0]:
-- Simple count of all patent records
SELECT COUNT(*) AS total_patents
FROM patent_data.patents;

## Query 2: Top 5 Patents with Most Claims


In [0]:
SELECT p.publication_number, 
       p.invention_title,
       COUNT(c.claim_id) AS claim_count
FROM patent_data.patents p
JOIN patent_data.claims_individual c
  ON p.publication_number = c.publication_number
GROUP BY p.publication_number, p.invention_title
ORDER BY claim_count DESC
LIMIT 5;

## Query 3: Top 10 CPC Section & Class Combinations with Highest Average Claims

In [0]:
WITH patent_claims AS (
    SELECT 
        p.cpc_section,
        p.cpc_class,
        p.publication_number,
        COUNT(c.claim_id) AS claim_count
    FROM patent_data.patents p
    JOIN patent_data.claims_individual c
      ON p.publication_number = c.publication_number
    GROUP BY p.cpc_section, p.cpc_class, p.publication_number
)
SELECT 
    cpc_section,
    cpc_class,
    SUM(claim_count) AS total_claims,
    COUNT(DISTINCT publication_number) AS total_patents,
    bround(SUM(claim_count) / COUNT(DISTINCT publication_number), 1) AS avg_claims_per_patent
FROM patent_claims
GROUP BY cpc_section, cpc_class
HAVING COUNT(DISTINCT publication_number) >= 10
ORDER BY avg_claims_per_patent DESC
LIMIT 10;

## Query 4: Top 5 Applicant Organizations by Average Claim Count


In [0]:
WITH applicant_patents AS (
    SELECT 
        a.applicant_name,
        a.publication_number
    FROM patent_data.applicants_individual a
    WHERE a.applicant_orgname IS NOT NULL  -- Focus on organizations only
    GROUP BY a.applicant_name, a.publication_number
),
inventor_counts AS (
    SELECT 
        ap.applicant_name,
        ap.publication_number,
        COUNT(DISTINCT i.inventor_name) AS inventor_count
    FROM applicant_patents ap
    LEFT JOIN patent_data.inventors_individual i
        ON ap.publication_number = i.publication_number
    GROUP BY ap.applicant_name, ap.publication_number
),
claim_counts AS (
    SELECT 
        ap.applicant_name,
        ap.publication_number,
        COUNT(c.claim_id) AS claim_count
    FROM applicant_patents ap
    JOIN patent_data.claims_individual c
        ON ap.publication_number = c.publication_number
    GROUP BY ap.applicant_name, ap.publication_number
),
applicant_stats AS (
    SELECT 
        ic.applicant_name,
        COUNT(DISTINCT ic.publication_number) AS patent_count,
        AVG(ic.inventor_count) AS avg_inventors_per_patent,
        SUM(cc.claim_count) AS total_claims,
        SUM(cc.claim_count) / COUNT(DISTINCT ic.publication_number) AS avg_claims_per_patent
    FROM inventor_counts ic
    JOIN claim_counts cc 
        ON ic.applicant_name = cc.applicant_name AND ic.publication_number = cc.publication_number
    GROUP BY ic.applicant_name
    HAVING COUNT(DISTINCT ic.publication_number) >= 3
)
SELECT 
    applicant_name,
    patent_count,
    bround(avg_claims_per_patent, 1) AS avg_claims_per_patent,
    bround(avg_inventors_per_patent, 1) AS avg_inventors_per_patent
FROM applicant_stats
ORDER BY avg_claims_per_patent DESC
LIMIT 5;

## Query 5: Top 10 Patents with Keyword "battery" in Abstract or Claims

In [0]:
WITH abstract_matches AS (
    SELECT 
        publication_number,
        CASE 
            WHEN lower(abstract) LIKE '%battery%' THEN 1
            ELSE 0
        END AS abstract_contains_battery
    FROM patent_data.patents
),
claim_matches AS (
    SELECT 
        publication_number,
        SUM(CASE 
            WHEN lower(claim_text) LIKE '%battery%' THEN 1
            ELSE 0
        END) AS battery_claim_count
    FROM patent_data.claims_individual
    GROUP BY publication_number
),
combined_matches AS (
    SELECT 
        p.publication_number,
        p.invention_title,
        COALESCE(am.abstract_contains_battery, 0) AS abstract_contains_battery,
        COALESCE(cm.battery_claim_count, 0) AS battery_claim_count
    FROM patent_data.patents p
    LEFT JOIN abstract_matches am ON p.publication_number = am.publication_number
    LEFT JOIN claim_matches cm ON p.publication_number = cm.publication_number
    WHERE COALESCE(am.abstract_contains_battery, 0) = 1 OR COALESCE(cm.battery_claim_count, 0) > 0
)
SELECT 
    publication_number,
    invention_title,
    abstract_contains_battery,
    battery_claim_count,
    (abstract_contains_battery + battery_claim_count) AS total_battery_mentions
FROM combined_matches
ORDER BY total_battery_mentions DESC
LIMIT 10;